# Import Library

In [24]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict
from utils.preprocessing import cleaningText
from utils.preprocessing import casefoldingText
from utils.preprocessing import stemmingText
from utils.preprocessing import fix_slangwords
from utils.preprocessing import tokenize
from utils.preprocessing import filteringText
from utils.preprocessing import tokenizingText
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.preprocessing import LabelEncoder
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

# Loading Dataset

In [2]:
# Ubah URL menjadi raw URL GitHub
chatbot_url = "https://raw.githubusercontent.com/novaandini/arkanesia/refs/heads/main/data/chatbot.csv"

# Baca CSV (ganti 'nama_file.csv' dengan file yang diunggah)
chatbot_df = pd.read_csv(chatbot_url)

# Cek kolom yang ada
print(chatbot_df.head())

                                         Pertanyaan       Intent  \
0  Apa tempat wisata alam terbaik di [nama_daerah]?  wisata_alam   
1  Apa tempat wisata alam terbaik di [nama_daerah]?  wisata_alam   
2  Apa tempat wisata alam terbaik di [nama_daerah]?  wisata_alam   
3  Apa tempat wisata alam terbaik di [nama_daerah]?  wisata_alam   
4  Apa tempat wisata alam terbaik di [nama_daerah]?  wisata_alam   

                                             Jawaban  
0  Di [nama_daerah], Anda bisa mengunjungi [rekom...  
1  [Nama_daerah] memiliki banyak tempat wisata al...  
2  Tempat wisata alam terbaik di [nama_daerah] te...  
3  [Nama_daerah] adalah surga bagi pecinta alam, ...  
4  Jika Anda mencari wisata alam di [nama_daerah]...  


## Explore Chatbot Data

In [3]:
chatbot_df.duplicated().sum()

np.int64(0)

In [4]:
chatbot_df.isnull().sum()

Pertanyaan    0
Intent        0
Jawaban       0
dtype: int64

In [5]:
chatbot_df.describe()

,Pertanyaan,Intent,Jawaban
count,1100,1100,1100
unique,110,14,140
top,Apa tempat wisata alam terbaik di [nama_daerah]?,wisata_alam,"Di [nama_daerah], Anda bisa mengunjungi [rekom..."
freq,10,100,10


In [6]:
chatbot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Pertanyaan  1100 non-null   object
 1   Intent      1100 non-null   object
 2   Jawaban     1100 non-null   object
dtypes: object(3)
memory usage: 25.9+ KB


# Preprocessing Text

## Cleaning Text

In [7]:
def protect_placeholders(text):
    text = text.replace('[nama_daerah]', '__NAMA_DAERAH__')
    for i in range(1, 5):
        text = text.replace(f'[rekomendasi_{i}]', f'__REK{i}__')
    return text

chatbot_cleaned_df = pd.DataFrame()
chatbot_cleaned_df['question'] = chatbot_df['Pertanyaan']
chatbot_cleaned_df['intent'] = chatbot_df['Intent']
chatbot_cleaned_df['answer'] = chatbot_df['Jawaban']
chatbot_cleaned_df.head()

,question,intent,answer
0,Apa tempat wisata alam terbaik di [nama_daerah]?,wisata_alam,"Di [nama_daerah], Anda bisa mengunjungi [rekom..."
1,Apa tempat wisata alam terbaik di [nama_daerah]?,wisata_alam,[Nama_daerah] memiliki banyak tempat wisata al...
2,Apa tempat wisata alam terbaik di [nama_daerah]?,wisata_alam,Tempat wisata alam terbaik di [nama_daerah] te...
3,Apa tempat wisata alam terbaik di [nama_daerah]?,wisata_alam,"[Nama_daerah] adalah surga bagi pecinta alam, ..."
4,Apa tempat wisata alam terbaik di [nama_daerah]?,wisata_alam,Jika Anda mencari wisata alam di [nama_daerah]...


In [8]:
chatbot_cleaned_df['question'] = chatbot_cleaned_df['question'].apply(cleaningText)
chatbot_cleaned_df['answer'] = chatbot_cleaned_df['answer'].apply(cleaningText)

chatbot_cleaned_df.head()

,question,intent,answer
0,Apa tempat wisata alam terbaik di namadaerah,wisata_alam,Di namadaerah Anda bisa mengunjungi rekomendas...
1,Apa tempat wisata alam terbaik di namadaerah,wisata_alam,Namadaerah memiliki banyak tempat wisata alam ...
2,Apa tempat wisata alam terbaik di namadaerah,wisata_alam,Tempat wisata alam terbaik di namadaerah terma...
3,Apa tempat wisata alam terbaik di namadaerah,wisata_alam,Namadaerah adalah surga bagi pecinta alam deng...
4,Apa tempat wisata alam terbaik di namadaerah,wisata_alam,Jika Anda mencari wisata alam di namadaerah pa...


## Case Folding text

In [9]:
chatbot_cleaned_df['question'] = chatbot_cleaned_df['question'].apply(casefoldingText)
chatbot_cleaned_df['answer'] = chatbot_cleaned_df['answer'].apply(casefoldingText)
chatbot_cleaned_df.head()

,question,intent,answer
0,apa tempat wisata alam terbaik di namadaerah,wisata_alam,di namadaerah anda bisa mengunjungi rekomendas...
1,apa tempat wisata alam terbaik di namadaerah,wisata_alam,namadaerah memiliki banyak tempat wisata alam ...
2,apa tempat wisata alam terbaik di namadaerah,wisata_alam,tempat wisata alam terbaik di namadaerah terma...
3,apa tempat wisata alam terbaik di namadaerah,wisata_alam,namadaerah adalah surga bagi pecinta alam deng...
4,apa tempat wisata alam terbaik di namadaerah,wisata_alam,jika anda mencari wisata alam di namadaerah pa...


## Fix Slang Words

In [10]:
chatbot_cleaned_df['question'] = chatbot_cleaned_df['question'].apply(fix_slangwords)
# chatbot_cleaned_df['answer'] = chatbot_cleaned_df['answer'].apply(fix_slangwords)
chatbot_cleaned_df.head()

,question,intent,answer
0,apa tempat wisata alam terbaik di namadaerah,wisata_alam,di namadaerah anda bisa mengunjungi rekomendas...
1,apa tempat wisata alam terbaik di namadaerah,wisata_alam,namadaerah memiliki banyak tempat wisata alam ...
2,apa tempat wisata alam terbaik di namadaerah,wisata_alam,tempat wisata alam terbaik di namadaerah terma...
3,apa tempat wisata alam terbaik di namadaerah,wisata_alam,namadaerah adalah surga bagi pecinta alam deng...
4,apa tempat wisata alam terbaik di namadaerah,wisata_alam,jika anda mencari wisata alam di namadaerah pa...


## Stemming Text

In [11]:
chatbot_cleaned_df['question'] = chatbot_cleaned_df['question'].apply(stemmingText)
# chatbot_cleaned_df['answer'] = chatbot_cleaned_df['answer'].apply(stemmingText)
chatbot_cleaned_df.head()

,question,intent,answer
0,apa tempat wisata alam baik di namadaerah,wisata_alam,di namadaerah anda bisa mengunjungi rekomendas...
1,apa tempat wisata alam baik di namadaerah,wisata_alam,namadaerah memiliki banyak tempat wisata alam ...
2,apa tempat wisata alam baik di namadaerah,wisata_alam,tempat wisata alam terbaik di namadaerah terma...
3,apa tempat wisata alam baik di namadaerah,wisata_alam,namadaerah adalah surga bagi pecinta alam deng...
4,apa tempat wisata alam baik di namadaerah,wisata_alam,jika anda mencari wisata alam di namadaerah pa...


In [12]:
def restore_placeholders(text):
    text = text.replace('namadaerah', '[nama_daerah]')
    for i in range(1, 5):
        text = text.replace(f'rek{i}', f'[rekomendasi_{i}]')
    return text

# chatbot_cleaned_df['question'] = chatbot_cleaned_df['question'].apply(restore_placeholders)
# chatbot_cleaned_df['answer'] = chatbot_cleaned_df['answer'].apply(restore_placeholders)
# chatbot_cleaned_df.head()

## Encode Label Intent

In [13]:
label_encoder = LabelEncoder()
chatbot_cleaned_df['label'] = label_encoder.fit_transform(chatbot_cleaned_df['intent'])

# Simpan untuk nanti decoding hasil prediksi
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
for label, encoded in label_mapping.items():
    print(f"{label} → {encoded}")

aktivitas → 0
budget → 1
cuaca → 2
detail_wisata → 3
kuliner → 4
lokasi_wisata → 5
penginapan → 6
transportasi → 7
wisata_alam → 8
wisata_bahari → 9
wisata_budaya → 10
wisata_edukasi → 11
wisata_rekreasi → 12
wisata_sejarah → 13


In [14]:
chatbot_cleaned_df.head()

,question,intent,answer,label
0,apa tempat wisata alam baik di namadaerah,wisata_alam,di namadaerah anda bisa mengunjungi rekomendas...,8
1,apa tempat wisata alam baik di namadaerah,wisata_alam,namadaerah memiliki banyak tempat wisata alam ...,8
2,apa tempat wisata alam baik di namadaerah,wisata_alam,tempat wisata alam terbaik di namadaerah terma...,8
3,apa tempat wisata alam baik di namadaerah,wisata_alam,namadaerah adalah surga bagi pecinta alam deng...,8
4,apa tempat wisata alam baik di namadaerah,wisata_alam,jika anda mencari wisata alam di namadaerah pa...,8


## Filtering Text

In [15]:
chatbot_cleaned_df['question_filtered'] = chatbot_cleaned_df['question'].apply(tokenizingText)
chatbot_cleaned_df['question_filtered'] = chatbot_cleaned_df['question_filtered'].apply(filteringText)
# chatbot_cleaned_df['answer'] = chatbot_cleaned_df['answer'].apply(filteringText)
chatbot_cleaned_df.head()

,question,intent,answer,label,question_filtered
0,apa tempat wisata alam baik di namadaerah,wisata_alam,di namadaerah anda bisa mengunjungi rekomendas...,8,"[wisata, alam, namadaerah]"
1,apa tempat wisata alam baik di namadaerah,wisata_alam,namadaerah memiliki banyak tempat wisata alam ...,8,"[wisata, alam, namadaerah]"
2,apa tempat wisata alam baik di namadaerah,wisata_alam,tempat wisata alam terbaik di namadaerah terma...,8,"[wisata, alam, namadaerah]"
3,apa tempat wisata alam baik di namadaerah,wisata_alam,namadaerah adalah surga bagi pecinta alam deng...,8,"[wisata, alam, namadaerah]"
4,apa tempat wisata alam baik di namadaerah,wisata_alam,jika anda mencari wisata alam di namadaerah pa...,8,"[wisata, alam, namadaerah]"


## Tokenizing Text

In [16]:
# chatbot_cleaned_df['question'] = chatbot_cleaned_df['question'].apply(tokenizingText)
# chatbot_cleaned_df['answer'] = chatbot_cleaned_df['answer'].apply(tokenizingText)

dataset = Dataset.from_pandas(chatbot_cleaned_df)
tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset[0]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map: 100%|██████████| 1100/1100 [00:00<00:00, 7617.73 examples/s]


{'question': 'apa tempat wisata alam baik di namadaerah',
 'intent': 'wisata_alam',
 'answer': 'di namadaerah anda bisa mengunjungi rekomendasi1 rekomendasi2 dan rekomendasi3 yang terkenal dengan pemandangan alam yang menakjubkan jangan lewatkan juga rekomendasi4 untuk pengalaman yang tak terlupakan',
 'label': 8,
 'question_filtered': ['wisata', 'alam', 'namadaerah'],
 'input_ids': [2,
  387,
  515,
  1223,
  668,
  342,
  26,
  712,
  10224,
  3,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 'labels': 8}

# Modeling

## Split Dataset

In [17]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

## Load Pretrained Model

In [18]:
num_labels = len(chatbot_cleaned_df['label'].unique())

model = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1",
    num_labels=num_labels
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training Argument

In [19]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01
)

c:\Users\ASUS\Desktop\arkanesia-dataset\arkanesia_env\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Training Setup

In [25]:
def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_8504\3151952263.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Train The Model

In [26]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.069657,1.000000
2,No log,0.011692,1.000000
3,No log,0.008111,1.000000
4,No log,0.007292,1.000000


TrainOutput(global_step=440, training_loss=0.2718481410633434, metrics={'train_runtime': 974.106, 'train_samples_per_second': 3.614, 'train_steps_per_second': 0.452, 'total_flos': 28945334415360.0, 'train_loss': 0.2718481410633434, 'epoch': 4.0})